In [1]:
from pyscf import gto, scf, cc

a = 2 # bond length in a cluster
d = 10 # distance between each cluster
unit = 'b' # unit of length
na = 2  # size of a cluster (monomer)
nc = 1 # set as integer multiple of monomers
spin = 0 # spin per monomer
frozen = 0 # frozen orbital per monomer
elmt = 'H'
basis = 'sto6g'
atoms = ""
for n in range(nc*na):
    shift = ((n - n % na) // na) * (d-a)
    atoms += f"{elmt} {n*a+shift:.5f} 0.00000 0.00000 \n"

mol = gto.M(atom=atoms,basis=basis,spin=spin*nc,unit=unit,verbose=4)
mol.build()

mf = scf.RHF(mol)
mf.kernel()

nfrozen = nc*frozen

mycc = cc.CCSD(mf,frozen=nfrozen)
mycc.kernel()

System: uname_result(system='Linux', node='yichi-thinkpad', release='4.4.0-26100-Microsoft', version='#7309-Microsoft Fri Jan 01 08:00:00 PST 2016', machine='x86_64')  Threads 12
Python 3.10.16 | packaged by conda-forge | (main, Dec  5 2024, 14:16:10) [GCC 13.3.0]
numpy 1.24.3  scipy 1.14.1  h5py 3.12.1
Date: Mon Dec 15 00:12:13 2025
PySCF version 2.8.0
PySCF path  /home/yichi/research/software/lno_pyscf
GIT HEAD (branch master) ef75f4190e4de208685670651dc6c467f72b6794

[ENV] PYSCF_EXT_PATH /home/yichi/research/software/pyscf
[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 2
[INPUT] num. electrons = 2
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 0
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = b
[INPUT] Symbol           X                Y                Z      unit          X                Y                Z       unit  Magmom
[INPUT]  1 H      0.000000000000   0.000000000000   0.000000000000 AA    0.000000000000   0.000000000000   0.000000000

(-0.039641400885220474, array([[-2.45726703e-16]]), array([[[[-0.19736585]]]]))

In [66]:
t1 = 10*mycc.t1
print(t1.shape)
print(t1)
eccsd = mycc.energy(t1, 0*mycc.t2)
print("CCSD correlation energy:", eccsd)

(1, 3)
[[ 2.38158128e-15  1.49976998e-01 -7.00573315e-16]]
CCSD correlation energy: 0.002054920397046801


In [67]:
import numpy as np
from jax import scipy as jsp
norb =  mf.mol.nao - nfrozen
nocc = t1.shape[0]
t1_full = np.zeros((norb, norb))
t1_full[:nocc, nocc:] = t1
print(t1_full)
exp_t1 = jsp.linalg.expm(t1_full)
exp_mt1 = jsp.linalg.expm(-t1_full)
print(exp_t1)
print(exp_mt1)

[[ 0.00000000e+00  2.38158128e-15  1.49976998e-01 -7.00573315e-16]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]]
[[ 1.00000000e+00  2.38158128e-15  1.49976998e-01 -7.00573315e-16]
 [ 0.00000000e+00  1.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
[[ 1.00000000e+00 -2.38158128e-15 -1.49976998e-01  7.00573315e-16]
 [ 0.00000000e+00  1.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [ ]:
options = {'n_eql': 3,
           'n_prop_steps': 50,
            'n_ene_blocks': 5,
            'n_sr_blocks': 5,
            'n_blocks': 10,
            'n_walkers': 1,
            'seed': 2,
            'walker_type': 'rhf',
            'trial': 'rhf',
            'dt':0.005,
            'free_projection':False,
            'ad_mode':None,
            'use_gpu': False,
            }

from ad_afqmc.lno_afqmc.ccsd_pt2 import lno_afqmc
nocc = mol.nelectron // 2
prjlo = np.eye(nocc)
lno_afqmc.prep_afqmc(mf,mf.mo_coeff,mycc.t1,mycc.t2,[],prjlo,options,chol_cut=1e-5)

ImportError: cannot import name 'hamiltonian' from 'ad_afqmc.lno_afqmc' (/home/yichi/research/software/cs_afqmc/ad_afqmc/lno_afqmc/__init__.py)

In [4]:
import pickle
option_file='options.bin'
with open(option_file, 'wb') as f:
    pickle.dump(options, f)

In [3]:
from ad_afqmc.prop_unrestricted import prop_unrestricted
ham_data, ham, prop, trial, wave_data, sampler, observable, options, _ = (
    prop_unrestricted._prep_afqmc())

# Hostname: yichi-thinkpad
# System Type: Linux
# Machine Type: x86_64
# Processor: x86_64
# Number of MPI ranks: 1
#
# norb: 4
# nelec: (1, 1)
#
# n_eql: 3
# n_prop_steps: 50
# n_ene_blocks: 5
# n_sr_blocks: 5
# n_blocks: 10
# n_walkers: 1
# seed: 2
# walker_type: rhf
# trial: rhf
# dt: 0.005
# free_projection: False
# use_gpu: False
# n_exp_terms: 6
# orbital_rotation: True
# do_sr: True
# symmetry: False
# save_walkers: False
# ene0: 0.0
# n_batch: 1
# LNO: False
# orbE: 0
# maxError: 0.001
#


In [148]:
from jax import random
import jax.numpy as jnp
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

### initialize propagation
seed = options["seed"]
init_walkers = None
trial_rdm1 = trial.get_rdm1(wave_data)
if "rdm1" not in wave_data:
    wave_data["rdm1"] = trial_rdm1
ham_data = ham.build_measurement_intermediates(ham_data, trial, wave_data)
ham_data = ham.build_propagation_intermediates(ham_data, prop, trial, wave_data)
h0 = ham_data['h0']

prop_data = prop.init_prop_data(trial, wave_data, ham_data, init_walkers)
if jnp.abs(jnp.sum(prop_data["overlaps"])) < 1.0e-6:
    raise ValueError(
        "Initial overlaps are zero. Pass walkers with non-zero overlap."
    )
prop_data["key"] = random.PRNGKey(seed + rank)

prop_data["overlaps"] = trial.calc_overlap(prop_data["walkers"], wave_data)
prop_data["n_killed_walkers"] = 0
prop_data["pop_control_ene_shift"] = prop_data["e_estimate"]
print(prop_data['e_estimate'])
# converged SCF energy = -1.08381072341461

-1.083810723414608


In [149]:
walker = prop_data['walkers'][0]
trial._calc_energy_restricted(walker,ham_data,wave_data)

Array(-1.08381072+0.j, dtype=complex128)

In [150]:
exp_walker = exp_t1.T @ walker
e_test = trial._calc_energy_restricted(exp_walker,ham_data,wave_data)
e_ref = mf.e_tot + eccsd
print(e_test, e_ref)

(-1.0817558030175616+0j) -1.0817558030175622


In [ ]:
import opt_einsum as oe
nocc = trial.nelec[0]
norb = trial.norb
h1_bar = exp_t1 @ ham_data['h1'][0] @ exp_mt1
chol_bar = jnp.einsum('pr,grs,sq->gpq', exp_t1, chol, exp_mt1)
ham_data["rot_h1_bar"] = wave_data["mo_coeff"].T.conj() @ h1_bar
ham_data["rot_chol_bar"] = oe.contract(
    "ip,gpq->giq", wave_data["mo_coeff"].T.conj(),
    chol_bar, backend="jax")
jeff = oe.contract('giq,gjj->iq', ham_data['rot_chol_bar'], 
                   chol_bar[:,:nocc,:nocc], backend="jax")
keff = oe.contract('gij,gjq->iq', ham_data['rot_chol_bar'][:,:,:nocc],
                   chol_bar[:,:nocc,:], backend="jax")
ham_data['rot_fock_bar'] = ham_data["rot_h1_bar"] + 2 * jeff - keff

In [ ]:
import opt_einsum as oe
from jax import vmap

def _calc_energy_ref(trial, walker, ham_data, trial_coeff):
    h0, h1 = ham_data["h0"], ham_data["h1"][0]
    chol = ham_data["chol"].reshape(-1, trial.norb, trial.norb)
    gf = (walker.dot(jnp.linalg.inv(trial_coeff.T @ walker)) @ trial_coeff.T).T
    ene1 = 2.0 * jnp.sum(gf * h1)
    f = oe.contract("gij,jk->gik", chol, gf.T, backend="jax")
    c = vmap(jnp.trace)(f)
    exc = jnp.sum(vmap(lambda x: x * x.T)(f))
    ene2 = 2.0 * jnp.sum(c * c) - exc
    return h0 + ene1 + ene2

def _calc_ecorr_bar(trial, walker, ham_data, wave_data):
    ''' calculate the correlation energy of the Hamiltonian
        transformed by exp(T1^dagger)
        ecorr_bar = <psi_0|H_bar|walker_bar>/<psi_0|walker_bar> - <psi_0|H_bar|psi_0>
        |walker_bar> = exp(T1^dagger) |walker>
        H_bar = exp(T1^dagger) H exp(-T1^dagger)
        |psi_0> is the mean-field solution of H
    '''
    walker = exp_t1 @ walker
    nocc = trial.nelec[0]
    rot_fock = ham_data['rot_fock_bar']
    rot_chol = ham_data['rot_chol_bar']
    gf = trial._calc_green(walker, wave_data)
    e1 = oe.contract('ia,ia->',gf[:nocc,nocc:],
                     rot_fock[:nocc,nocc:], backend="jax") * 2
    lg = oe.contract('gia,ka->gik', rot_chol[:,:nocc,nocc:],
                     gf[:nocc,nocc:], backend="jax")
    tr_lg = oe.contract('gii->g', lg, backend="jax")
    e2 = oe.contract('g,g->', tr_lg, tr_lg, backend="jax")*2 \
        - oe.contract('gij,gji->',lg,lg, backend="jax")
    e_corr = e1 + e2
    return jnp.real(e_corr)

In [2]:
import numpy as np
np.count_nonzero([1,2,3,4,5,0])

5

In [153]:
exp_trial_mo = exp_t1.T @ wave_data['mo_coeff']
e1 = jnp.real(_calc_energy_ref(trial, walker, ham_data, wave_data['mo_coeff']))
e2 = jnp.real(_calc_energy_ref(trial, walker, ham_data, exp_trial_mo))
print(e1, e2)

-1.083810723414608 -1.0817558030175614


In [ ]:
import numpy as np
walker_r = np.random.randn(*walker.shape)
e_test = jnp.real(_calc_energy_ref(trial, walker, ham_data, exp_trial_mo
                 )+_calc_ecorr_nc(trial, walker_r, ham_data, wave_data))
e_ref = _calc_energy_ref(trial, walker_r, ham_data, exp_trial_mo)
print(e_test, e_ref)

-0.9983436363589793 -0.9983436363589802
